In [1]:
%%capture
%load_ext autoreload
%autoreload 2

In [62]:
from src.repository import DatabaseClient, to_table, \
    CityRepository, CountryRepository, RegionRepository, UserRepository, \
    City, Country, Region, User

from src.entity_loader import EntityLoader, \
    CityMapper, RegionMapper, UserMapper, CountryMapper, \
    CountryFilter, CityFilter

### Database client

In [74]:
USERNAME = "neo4j"
PASSWORD = USERNAME
URL = "bolt://localhost:7687"

client = DatabaseClient(URL, USERNAME, PASSWORD)

### Repositories

In [75]:
cityRepository = CityRepository(client)

countryRepository = CountryRepository(client)

regionRepository = RegionRepository(client)

userRepository = UserRepository(client)

### Remove all nodes

In [5]:
# client.run("MATCH (n:City) DETACH DELETE n")
client.run("MATCH (n:User) DETACH DELETE n")
client.run("MATCH (n:Country) DETACH DELETE n")
client.run("MATCH (n:Region) DETACH DELETE n")
client.run("MATCH (n:Persons) DETACH DELETE n")
client.run("MATCH (n:ProductSearch) DETACH DELETE n")
client.run("MATCH (n:Purchase) DETACH DELETE n")
client.run("MATCH (n:ProductReservation) DETACH DELETE n")

### Cities

In [6]:
cities = list(EntityLoader.load_filter_map('database/cities.json', CityFilter(), CityMapper()))
len(cities)

120342

In [7]:
# cityRepository.upsert(cities)

### Countries

In [8]:
countries = list(EntityLoader.load_filter_map('database/countries.json', CountryFilter(), CountryMapper()))
len(countries)

199

In [10]:
countryRepository.upsert(countries)

### Global Regions

In [11]:
regions = list(EntityLoader.load_map('database/regions.json', RegionMapper()))
len(regions)

7

In [12]:
regionRepository.upsert(regions)

### Users

In [13]:
users = list(EntityLoader.load_map('database/users.json', UserMapper()))
len(users)

7

In [14]:
users[0]

User(id=1, email='adrian.marino@almundo.com', realm='ALMUNDO-ALMUNDO-EMPLOYEE')

In [15]:
userRepository.upsert(users)

### City belong to Country

In [16]:
def city_belong_to_country(relation):
    client.run("""
        MATCH
            (city: City {iata_code: "%s"}),
            (country: Country {code: "%s"})
        WITH
            city,
            country
        MERGE
            (city)-[:BELONG_TO]->(country)
    """ %(relation['city'],relation['country'])
)

In [17]:
city_country = list(EntityLoader.load('database/city-country.json'))
len(city_country)

7

In [18]:
[city_belong_to_country(relation) for relation in city_country]

[None, None, None, None, None, None, None]

### Country belong to Region

In [19]:
def country_belong_to_region(relation):
    client.run("""
        MATCH
            (country: Country {code: "%s"}),
            (region: Region {code: "%s"})
        WITH
            country,
            region
        MERGE
            (country)-[:BELONG_TO]->(region)
    """ %(relation['country'],relation['region'])
)

In [20]:
country_region = list(EntityLoader.load('database/country-region.json'))
len(country_region)

8

In [21]:
[country_belong_to_region(relation) for relation in country_region]

[None, None, None, None, None, None, None, None]

### Product Searchs

In [22]:
client.run("""
MATCH
    (BUE: City {iata_code: "BUE"}), 
    (MIA: City {iata_code: "MIA"}),
    (adrian: User {email: "adrian.marino@almundo.com"})
WITH
    BUE, MIA, adrian
CREATE
    (search1:ProductSearch {
        product_type: 'FLIGHT',
        from: datetime("2021-01-01T20:00:0"),
        to: datetime("2021-01-15T20:00:0")
    }),
    (search1)-[:FROM_LOCATION]->(BUE), (search1)-[:TO_LOCATION]->(MIA),
    (search1)-[:FOR]->(:Persons{adults:2, children: 2, infants: 1}),
    (adrian)-[:SEARCHED {datetime: datetime("2020-08-15T20:00:00")}]->(search1)
""")

In [23]:
client.run("""
MATCH
    (BUE: City {iata_code: "BUE"}),
    (BCN: City {iata_code: "BCN"}),
    (adrian: User {email: "adrian.marino@almundo.com"})
WITH
    BUE, BCN, adrian
CREATE
    (search2:ProductSearch {
        product_type: 'FLIGHT',
        from: datetime("2021-01-01T20:00:0"),
        to: datetime("2021-01-15T20:00:0")
    }),
    (search2)-[:FROM_LOCATION]->(BUE), (search2)-[:TO_LOCATION]->(BCN),
    (search2)-[:FOR]->(:Persons{adults:2, children: 2, infants: 1}),
    (adrian)-[:SEARCHED{datetime:datetime("2020-08-15T20:10:00")}]->(search2)

""")

In [24]:
client.run("""
MATCH
    (MIA: City {iata_code: "MIA"}),
    (adrian: User {email: "adrian.marino@almundo.com"})
WITH
    MIA, adrian
CREATE
    (search3: ProductSearch {
        product_type: 'HOTEL',
        from: datetime("2021-01-01T20:00:0"),
        to: datetime("2021-01-15T20:00:0")
    }),
    (search3)-[:IN_LOCATION]->(MIA),
    (search3)-[:FOR]->(:Persons {adults:2, children: 2, infants: 1}),
    (adrian)-[:SEARCHED{datetime:datetime("2020-08-15T22:00:00")}]->(search3)
""")

In [25]:
client.run("""
MATCH
    (BCN: City {iata_code: "BCN"}),
    (adrian: User {email: "adrian.marino@almundo.com"})
WITH
    BCN, adrian
CREATE
    (search4: ProductSearch {
        product_type: 'HOTEL',
        from: datetime("2021-01-01T20:00:0"),
        to: datetime("2021-01-15T20:00:0")
    }),
    (search4)-[:LOCATION]->(BCN),
    (search4)-[:FOR]->(:Persons {adults:2, children: 2, infants: 1}),
    (adrian)-[:SEARCHED{datetime:datetime("2020-08-15T22:00:00")}]->(search4)
""")

In [26]:
client.run("""
MATCH
    (COR: City {iata_code: "COR"}),
    (SLA: City {iata_code: "SLA"}),
    (juanManuel {email: "juanmanuel.bouvier@almundo.com"})
WITH
    COR, SLA, juanManuel
CREATE
    (search5:ProductSearch {
        product_type: 'FLIGHT',
        from: datetime("2021-01-01T20:00:0"),
        to: datetime("2021-01-15T20:00:0")
    }),
    (search5)-[:FROM_LOCATION]->(COR), 
    (search5)-[:TO_LOCATION]->(SLA),
    (search5)-[:FOR]->(:Persons{adults:2}),
    (juanManuel)-[:SEARCHED {datetime: datetime("2020-08-15T20:00:00")}]->(search5)

""")

In [27]:
client.run("""
MATCH 
    (COR: City {iata_code: "COR"}),
    (SLA: City {iata_code: "SLA"}),
    (juanManuel {email: "juanmanuel.bouvier@almundo.com"})
WITH
    COR, SLA, juanManuel
CREATE
    (search6:ProductSearch {
        product_type: 'FLIGHT',
        from: datetime("2021-01-01T20:00:0"),
        to: datetime("2021-01-15T20:00:0")
    }),
    (search6)-[:FROM_LOCATION]->(COR),
    (search6)-[:TO_LOCATION]->(SLA),
    (search6)-[:FOR]->(:Persons{adults:2}),
    (juanManuel)-[:SEARCHED {datetime: datetime("2020-08-15T21:00:00")}]->(search6)

""")

In [28]:
client.run("""
MATCH
    (BUE: City {iata_code: "BUE"}),
    (BHD: City {iata_code: "RIO"}),
    (sofia: User {email: "sofia.massobrio@almundo.com"})
WITH
    BUE, BHD, sofia
CREATE
    (search7:ProductSearch {
        product_type: 'FLIGHT',
        from: datetime("2021-01-01T20:00:0"),
        to: datetime("2021-01-15T20:00:0")
    }),
    (search7)-[:FROM_LOCATION]->(BUE),
    (search7)-[:TO_LOCATION]->(BHD),
    (search7)-[:FOR]->(:Persons{adults:2}),
    (sofia)-[:SEARCHED {datetime: datetime("2020-08-15T21:00:00")}]->(search7)
""")

In [29]:
client.run("""
MATCH
    (BUE: City {iata_code: "BUE"}), 
    (BHD: City {iata_code: "RIO"}),
    (sofia: User {email: "sofia.massobrio@almundo.com"})
WITH
    BUE, BHD, sofia
CREATE
    (search8:ProductSearch {
        product_type: 'FLIGHT',
        from: datetime("2021-01-01T20:00:0"),
        to: datetime("2021-01-15T20:00:0")
    }),
    (search8)-[:FROM_LOCATION]->(BHD),
    (search8)-[:TO_LOCATION]->(BUE),
    (search8)-[:FOR]->(:Persons{adults:2}),
    (sofia)-[:SEARCHED {datetime: datetime("2020-08-15T21:00:00")}]->(search8)
""")

In [30]:
client.run("""
MATCH
    (BUE: City {iata_code: "BUE"}),
    (MIA: City {iata_code: "MIA"}),
    (sofia: User {email: "sofia.massobrio@almundo.com"})
WITH
    BUE, MIA, sofia
CREATE
    (search9:ProductSearch {
        product_type: 'FLIGHT',
        from: datetime("2021-01-01T20:00:0"),
        to: datetime("2021-01-15T20:00:0")
    }),
    (search9)-[:FROM_LOCATION]->(BUE), 
    (search9)-[:TO_LOCATION]->(MIA),
    (search9)-[:FOR]->(:Persons{adults:2}),
    (sofia)-[:SEARCHED {datetime: datetime("2020-08-15T21:00:00")}]->(search9)
""")

In [31]:
client.run("""
MATCH
    (COL: City {iata_code: "COL"}),
    (SLA: City {iata_code: "SLA"}),
    (sofia: User {email: "sofia.massobrio@almundo.com"})
WITH
    COL, SLA, sofia
CREATE
    (search10:ProductSearch {
        product_type: 'FLIGHT',
        from: datetime("2021-01-01T20:00:0"),
        to: datetime("2021-01-15T20:00:0")
    }),
    (search10)-[:FROM_LOCATION]->(SLA), (search10)-[:TO_LOCATION]->(COL),
    (search10)-[:FOR]->(:Persons{adults:2}),
    (sofia)-[:SEARCHED {datetime: datetime("2020-08-15T21:00:00")}]->(search10)
""")

In [32]:
client.run("""
MATCH
    (COR: City {iata_code: "COR"}),
    (SLA: City {iata_code: "SLA"}),
    (kevo: User {email: "kevin.klaus@almundo.com"})
WITH
    COR, SLA, kevo
CREATE
    (search11:ProductSearch {
        product_type: 'FLIGHT',
        from: datetime("2021-01-01T20:00:0"),
        to: datetime("2021-01-15T20:00:0")
    }),
    (search11)-[:FROM_LOCATION]->(COR), (search11)-[:TO_LOCATION]->(SLA),
    (search11)-[:FOR]->(:Persons{adults:2}),
    (kevo)-[:SEARCHED {datetime: datetime("2020-08-15T21:00:00")}]->(search11)
""")

In [33]:
client.run("""
MATCH
    (BUE: City {iata_code: "BUE"}),
    (MIA: City {iata_code: "MIA"}),
    (kevo: User {email: "kevin.klaus@almundo.com"})
WITH
    BUE, MIA, kevo
CREATE
    (search12:ProductSearch {
        product_type: 'FLIGHT',
        from: datetime("2021-01-01T20:00:0"),
        to: datetime("2021-01-15T20:00:0")
    }),
    (search12)-[:FROM_LOCATION]->(BUE),
    (search12)-[:TO_LOCATION]->(MIA),
    (search12)-[:FOR]->(:Persons{adults:2}),
    (kevo)-[:SEARCHED {datetime: datetime("2020-08-15T21:00:00")}]->(search12)
""")

In [34]:
client.run("""
MATCH
    (BCN: City {iata_code: "BCN"}),
    (facu:User {email: "facundo.etchanchu@almundo.com"})
WITH
    BCN, facu                                                   
CREATE
    (search13: ProductSearch {
        product_type: 'HOTEL',
        from: datetime("2021-01-01T20:00:0"),
        to: datetime("2021-01-15T20:00:0")
    }),
    (search13)-[:LOCATION]->(BCN),
    (search13)-[:FOR]->(:Persons {adults:2, children: 2, infants: 1}),
    (facu)-[:SEARCHED{datetime:datetime("2020-08-15T22:00:00")}]->(search13)
""")

In [35]:
client.run("""
MATCH
    (MIA: City {iata_code: "MIA"}),
    (facu:User {email: "facundo.etchanchu@almundo.com"})
WITH
    MIA, facu
CREATE
    (search14: ProductSearch {
        product_type: 'HOTEL',
        from: datetime("2021-01-01T20:00:0"),
        to: datetime("2021-01-15T20:00:0")
    }),
    (search14)-[:LOCATION]->(MIA),
    (search14)-[:FOR]->(:Persons {adults:2, children: 2, infants: 1}),
    (facu)-[:SEARCHED{datetime:datetime("2020-08-15T22:00:00")}]->(search14)
""")

In [36]:
client.run("""
MATCH
    (MIA: City {iata_code: "MIA"}),
    (kevo: User {email: "kevin.klaus@almundo.com"})
WITH
    MIA, kevo
CREATE
    (search15: ProductSearch {
        product_type: 'HOTEL',
        from: datetime("2021-01-01T20:00:0"),
        to: datetime("2021-01-15T20:00:0")
    }),
    (search15)-[:LOCATION]->(MIA),
    (search15)-[:FOR]->(:Persons {adults:2, children: 2, infants: 1}),
    (kevo)-[:SEARCHED{datetime:datetime("2020-08-15T22:00:00")}]->(search15)
""")

In [37]:
client.run("""
MATCH
    (COR: City {iata_code: "COR"}),
    (kevo: User {email: "kevin.klaus@almundo.com"})
WITH
    COR, kevo
CREATE
    (search16: ProductSearch {
        product_type: 'HOTEL',
        from: datetime("2021-01-01T20:00:0"),
        to: datetime("2021-01-15T20:00:0")
    }),
    (search16)-[:LOCATION]->(COR),
    (search16)-[:FOR]->(:Persons {adults:2, children: 2, infants: 1}),
    (kevo)-[:SEARCHED{datetime:datetime("2020-08-15T22:00:00")}]->(search16)
""")

In [38]:
client.run("""
MATCH
    (SLA: City {iata_code: "SLA"}),
    (kevo: User {email: "kevin.klaus@almundo.com"})
WITH
    SLA, kevo
CREATE
    (search17: ProductSearch {
        product_type: 'HOTEL',
        from: datetime("2021-01-01T20:00:0"),
        to: datetime("2021-01-15T20:00:0")
    }),
    (search17)-[:LOCATION]->(SLA),
    (search17)-[:FOR]->(:Persons {adults:2, children: 2, infants: 1}),
    (kevo)-[:SEARCHED{datetime:datetime("2020-08-15T22:00:00")}]->(search17)
""")

### Purchases

In [39]:
client.run("""
MATCH
    (BUE: City {iata_code: "BUE"}),
    (MIA: City {iata_code: "MIA"}),
    (adrian: User {email: "adrian.marino@almundo.com"})
WITH
    BUE, MIA, adrian
CREATE
    (reservation1: ProductReservation {
        product_type: 'FLIGHT',
        from: datetime("2021-01-01T20:00:0"),
        to: datetime("2021-01-15T20:00:0")
    }),
    (reservation1)-[:FROM_LOCATION]->(BUE), (reservation1)-[:TO_LOCATION]->(MIA),
    (reservation1)-[:FOR]->(:Persons {adults:2, children: 2, infants: 1}),
    (reservation2: ProductReservation {
        product_type: 'INSURANCE',
        from: datetime("2021-01-01T20:00:0"),
        to: datetime("2021-01-15T20:00:0")
    })-[:FOR]->(:Persons {adults:2, children: 2, infants: 1}),
    (adrian)-[:MAKE_A]->(purchase1: Purchase {datetime: datetime("2020-08-16T20:00:0")}),
    (purchase1)-[:CONTAINS]->(reservation1),
    (purchase1)-[:CONTAINS]->(reservation2)
""")

In [40]:
client.run("""
MATCH
    (MIA: City {iata_code: "MIA"}),
    (adrian: User {email: "adrian.marino@almundo.com"})
WITH
    MIA, adrian
CREATE
    (reservation3: ProductReservation {
        product_type: 'HOTEL',
        from: datetime("2021-01-01T20:00:0"),
        to: datetime("2021-01-15T20:00:0")
    }),
    (reservation3)-[:IN_LOCATION]->(MIA),
    (reservation3)-[:FOR]->(:Persons {adults:2, children: 2, infants: 1}), 
    (adrian)-[:MAKE_A]->(:Purchase {datetime: datetime("2020-08-16T20:00:0")})-[:CONTAINS]->(reservation3)

""")

In [41]:
client.run("""
MATCH 
    (COR: City {iata_code: "COR"}),
    (SLA: City {iata_code: "SLA"}),
    (juanManuel {email: "juanmanuel.bouvier@almundo.com"})
WITH
    COR, SLA, juanManuel
CREATE
    (reservation4: ProductReservation {
        product_type: 'FLIGHT',
        from: datetime("2021-01-01T20:00:0"),
        to: datetime("2021-01-15T20:00:0")
    }),
    (reservation4)-[:FROM_LOCATION]->(COR), (reservation4)-[:TO_LOCATION]->(SLA),
    (reservation4)-[:FOR]->(:Persons {adults:2, children: 2, infants: 1}),
    (juanManuel)-[:MAKE_A]->(:Purchase {datetime: datetime("2020-08-16T20:00:0")})-[:CONTAINS]->(reservation4)
""")

In [42]:
client.run("""

MATCH
    (BUE: City {iata_code: "BUE"}),
    (sofia: User {email: "sofia.massobrio@almundo.com"})
WITH
    BUE, sofia
CREATE
    (reservation5: ProductReservation {
        product_type: 'HOTEL',
        from: datetime("2021-01-01T20:00:0"),
        to: datetime("2021-01-15T20:00:0")
    }),
    (reservation5)-[:IN_LOCATION]->(BUE),
    (reservation5)-[:FOR]->(:Persons {adults:2}), 
    (sofia)-[:MAKE_A]->(:Purchase {datetime: datetime("2020-08-16T20:00:0")})-[:CONTAINS]->(reservation5)

""")

In [43]:
client.run("""
MATCH
    (BCN: City {iata_code: "BCN"}),
    (kevo: User {email: "kevin.klaus@almundo.com"})
WITH
    BCN, kevo
CREATE
    (reservation6: ProductReservation {
        product_type: 'HOTEL',
        from: datetime("2021-01-01T20:00:0"),
        to: datetime("2021-01-15T20:00:0")
    }),
    (reservation6)-[:IN_LOCATION]->(BCN),
    (reservation6)-[:FOR]->(:Persons {adults:2}), 
    (kevo)-[:MAKE_A]->(:Purchase {datetime: datetime("2020-08-16T20:00:0")})-[:CONTAINS]->(reservation6)
""")

In [44]:
client.run("""
MATCH
    (MIA: City {iata_code: "MIA"}),
    (facu:User {email: "facundo.etchanchu@almundo.com"})
WITH
    MIA, facu
CREATE
    (reservation7: ProductReservation {
        product_type: 'HOTEL',
        from: datetime("2021-01-01T20:00:0"),
        to: datetime("2021-01-15T20:00:0")
    }),
    (reservation7)-[:IN_LOCATION]->(MIA),
    (reservation7)-[:FOR]->(:Persons {adults:2}), 
    (facu)-[:MAKE_A]->(:Purchase {datetime: datetime("2020-08-16T20:00:0")})-[:CONTAINS]->(reservation7)
""")

In [45]:
client.run("""
MATCH
    (BUE: City {iata_code: "BUE"}),
    (facu:User {email: "facundo.etchanchu@almundo.com"})
WITH
    BUE, facu
CREATE
    (reservation8: ProductReservation {
        product_type: 'HOTEL',
        from: datetime("2022-01-01T20:00:0"),
        to: datetime("2022-01-15T20:00:0")
    }),
    (reservation8)-[:IN_LOCATION]->(BUE),
    (reservation8)-[:FOR]->(:Persons {adults:2}), 
    (facu)-[:MAKE_A]->(:Purchase {datetime: datetime("2020-08-16T20:00:0")})-[:CONTAINS]->(reservation8)
""")

In [46]:
client.run("""
MATCH
    (BCN: City {iata_code: "BCN"}),
    (facu: User {email: "facundo.etchanchu@almundo.com"})
WITH
    BCN, facu
CREATE
    (reservation9: ProductReservation {
        product_type: 'HOTEL',
        from: datetime("2022-01-01T20:00:0"),
        to: datetime("2022-01-15T20:00:0")
    }),
    (reservation9)-[:IN_LOCATION]->(BCN),
    (reservation9)-[:FOR]->(:Persons {adults:2}), 
    (facu)-[:MAKE_A]->(:Purchase {datetime: datetime("2020-08-16T20:00:0")})-[:CONTAINS]->(reservation9)                                      
""")

In [47]:
client.run("""
MATCH
    (MIA: City {iata_code: "MIA"}),
    (gonza:User {email: "gonzalo.prieto@almundo.com"})
WITH
    MIA, gonza
CREATE
    (reservation10: ProductReservation {
        product_type: 'HOTEL',
        from: datetime("2021-01-01T20:00:0"),
        to: datetime("2021-01-15T20:00:0")
    }),
    (reservation10)-[:IN_LOCATION]->(MIA),
    (reservation10)-[:FOR]->(:Persons {adults:2}), 
    (gonza)-[:MAKE_A]->(:Purchase {datetime: datetime("2020-08-16T20:00:0")})-[:CONTAINS]->(reservation10)
""")

In [48]:
client.run("""
MATCH
    (BUE: City {iata_code: "BUE"}),
    (gonza:User {email: "gonzalo.prieto@almundo.com"})
WITH
    BUE, gonza
CREATE
    (reservation11: ProductReservation {
        product_type: 'HOTEL',
        from: datetime("2022-01-01T20:00:0"),
        to: datetime("2022-01-15T20:00:0")
    }),
    (reservation11)-[:IN_LOCATION]->(BUE),
    (reservation11)-[:FOR]->(:Persons {adults:2}), 
    (gonza)-[:MAKE_A]->(:Purchase {datetime: datetime("2020-08-16T20:00:0")})-[:CONTAINS]->(reservation11)

""")

In [49]:
client.run("""
MATCH
    (BCN: City {iata_code: "BCN"}),
    (gonza:User {email: "gonzalo.prieto@almundo.com"})
WITH
    BCN, gonza
CREATE
    (reservation12: ProductReservation {
        product_type: 'HOTEL',
        from: datetime("2022-01-01T20:00:0"),
        to: datetime("2022-01-15T20:00:0")
    }),
    (reservation12)-[:IN_LOCATION]->(BCN),
    (reservation12)-[:FOR]->(:Persons {adults:2}), 
    (gonza)-[:MAKE_A]->(:Purchase {datetime: datetime("2020-08-16T20:00:0")})-[:CONTAINS]->(reservation12)
""")

In [80]:
client.run("""
MATCH
    (fromCity: City {iata_code: "BUE"}),
    (toCity: City {iata_code: "MIA"}),
    (user: User {email: "gonzalo.prieto@almundo.com"})

WITH
    fromCity,
    toCity,
    user

CREATE
    (reservation: ProductReservation {
        product_type: 'FLIGHT',
        from: datetime("2021-01-01T20:00:0"),
        to: datetime("2021-01-15T20:00:0")
    }),

    (reservation)-[:FROM_LOCATION]->(fromCity),
    (reservation)-[:TO_LOCATION]->(toCity),
    (reservation)-[:FOR]->(:Persons {adults:2}),

    (user)-[:MAKE_A]->(purchase: Purchase {datetime: datetime("2020-08-16T20:00:0")}),

    (purchase)-[:CONTAINS]->(reservation)
""")

### Consultas

1. Los Productos mas buscados por destino en lo ultimso 10 dias:

In [67]:
QUERY1 = """
MATCH   (:User)-[:MAKE_A]->
        (p:Purchase)-[:CONTAINS]->
        (r:ProductReservation)-[:IN_LOCATION|:TO_LOCATION]->
        (c:City)
WHERE
    p.datetime >= (datetime() - duration('P10D'))
RETURN
    r.product_type as product_type,
    c.name as destination,
    COUNT(*) as purchases
ORDER BY
    purchases DESC, 
    product_type ASC, 
    destination ASC
LIMIT 3
"""
columns=['product_type', 'destination', 'purchases']

client.query(lambda tx: to_table(tx.run(QUERY1), columns))

,product_type,destination,purchases
0,HOTEL,Barcelona,3
1,HOTEL,Buenos Aires,3
2,HOTEL,Miami,3


2. Los Productos mas comprados por destino en lo ultimso 10 dias:

In [68]:
QUERY2 = """
MATCH   (:User)-[s:SEARCHED]->
        (ps:ProductSearch)-[:IN_LOCATION|:TO_LOCATION]->
        (c:City)
WHERE
    s.datetime >= (datetime() - duration('P10D'))
RETURN
    ps.product_type as product_type, 
    c.name as destination,
    COUNT(*) as searches
ORDER BY
    searches DESC,
    product_type ASC,
    destination ASC
LIMIT 3
"""
columns=['product_type', 'destination', 'searches']

client.query(lambda tx: to_table(tx.run(QUERY2), columns))

,product_type,destination,searches
0,FLIGHT,Miami,3
1,FLIGHT,Salta,3
2,FLIGHT,Barcelona,1


3. Quiere ofreserle a un usuario los productos que compraron otros usuarios similares a este: 

In [84]:
email="adrian.marino@almundo.com"

QUERY3 = """
    MATCH
        (u:User {email: "%s"})
            -[:MAKE_A]->(up:Purchase)
            -[:CONTAINS]->(u_pr:ProductReservation)
            -[:TO_LOCATION|:IN_LOCATION]->(u_city:City),
        (ou:User)
            -[:MAKE_A]->(oup:Purchase)
            -[:CONTAINS]->(ou_pr:ProductReservation)
            -[:TO_LOCATION|:IN_LOCATION]->(ou_city:City)
    WHERE
        ou <> u
        AND ou_city <> u_city 
        AND up.datetime >= (datetime() - duration('P60D')) 
        AND oup.datetime >= (datetime() - duration('P60D'))
    WITH
        distinct(ou_pr) as product
    MATCH
        (product)-[:TO_LOCATION|:IN_LOCATION]->(city:City)
    WITH
        product.product_type as product_type,
        city.name as destination,
        REDUCE (sc=0, c in COLLECT(city) | sc+1 ) AS score
    RETURN
        product_type as product,
        destination,
        score
    ORDER BY
        score DESC
    LIMIT 10
""" % (email)


columns=['product', 'destination', 'score']
client.query(lambda tx: to_table(tx.run(QUERY3), columns))

,product,destination,score
0,HOTEL,Buenos Aires,3
1,HOTEL,Barcelona,3
2,FLIGHT,Salta,1
